## Kalshi Research - Paper Trading 

### Prashanth Bhaskara

## Loading Keys and Importing Libraries

In [ ]:
!pip install cryptography
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
import base64
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.exceptions import InvalidSignature
from pathlib import Path
import os
from cryptography.hazmat.backends import default_backend
import requests
import datetime

In [2]:
# Cell 1 — load KALSHI keys from .env in the same directory as this notebook


# --- Try to use python-dotenv if it's installed (recommended) ---
try:
    from dotenv import load_dotenv  # pip install python-dotenv
    env_path = Path.cwd() / ".env"   # assumes notebook is run from the same directory as .env
    if not env_path.exists():
        raise FileNotFoundError(f".env not found at: {env_path}")
    load_dotenv(dotenv_path=env_path, override=False)
except ImportError:
    # --- Fallback: minimal .env parser (no extra install needed) ---
    env_path = Path.cwd() / ".env"
    if not env_path.exists():
        raise FileNotFoundError(f".env not found at: {env_path}")

    for line in env_path.read_text().splitlines():
        line = line.strip()
        if not line or line.startswith("#") or "=" not in line:
            continue
        k, v = line.split("=", 1)
        k = k.strip()
        v = v.strip().strip('"').strip("'")
        os.environ.setdefault(k, v)  # don't overwrite if already set

# --- Read your keys into variables ---
public_key = os.getenv("KALSHI-ACCESS-KEY-DEMO")

missing = [k for k, v in {
    "KALSHI-ACCESS-KEY": public_key
}.items() if not v]

if missing:
    raise KeyError(f"Missing env var(s): {missing}. Check your .env formatting and key names.")

print("Loaded Kalshi keys ✅")  # intentionally not printing the secret values


Loaded Kalshi keys ✅


### Next, construct functions to be able to pull Kalshi data

In [27]:


def load_private_key_from_file(file_path):
    with open(file_path, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,  # or provide a password if your key is encrypted
            backend=default_backend()
        )
    return private_key


def sign_pss_text(private_key: rsa.RSAPrivateKey, text: str) -> str:
    message = text.encode('utf-8')
    try:
        signature = private_key.sign(
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.DIGEST_LENGTH
            ),
            hashes.SHA256()
        )
        return base64.b64encode(signature).decode('utf-8')
    except InvalidSignature as e:
        raise ValueError("RSA sign PSS failed") from e
    

def make_authenticated_request(path, method='GET', params=None):
    """Make an authenticated request to Kalshi demo API."""
    current_time = datetime.datetime.now()
    timestamp = current_time.timestamp()
    current_time_milliseconds = int(timestamp * 1000)
    timestamp_str = str(current_time_milliseconds)
    
    private_key = load_private_key_from_file('pbhaskarademo.txt')
    
    base_url = 'https://demo-api.kalshi.co'
    
    # Strip query parameters from path before signing
    path_without_query = path.split('?')[0]
    msg_string = timestamp_str + method + path_without_query
    sig = sign_pss_text(private_key, msg_string)
    
    headers = {
        'KALSHI-ACCESS-KEY': public_key,
        'KALSHI-ACCESS-SIGNATURE': sig,
        'KALSHI-ACCESS-TIMESTAMP': timestamp_str
    }
    
    if method == 'GET':
        response = requests.get(base_url + path, headers=headers, params=params)
    else:
        response = requests.post(base_url + path, headers=headers, json=params)
    
    return response

In [26]:

# === 1. Get Portfolio Balance ===
print("=" * 70)
print("PORTFOLIO BALANCE")
print("=" * 70)

balance_response = make_authenticated_request('/trade-api/v2/portfolio/balance')

if balance_response.status_code == 200:
    balance_data = balance_response.json()
    
    available_balance = balance_data.get('balance', 0) / 100  # Convert cents to dollars
    portfolio_value = balance_data.get('portfolio_value', 0) / 100
    updated_ts = balance_data.get('updated_ts', 0)
    
    if updated_ts:
        updated_time = datetime.datetime.fromtimestamp(updated_ts, tz=datetime.timezone.utc)
        try:
            from zoneinfo import ZoneInfo
            ET_TZ = ZoneInfo('America/New_York')
            updated_time_et = updated_time.astimezone(ET_TZ).strftime('%Y-%m-%d %H:%M:%S %Z')
        except:
            updated_time_et = updated_time.strftime('%Y-%m-%d %H:%M:%S UTC')
    else:
        updated_time_et = 'N/A'
    
    print(f"\n  Available Balance:    ${available_balance:,.2f}")
    print(f"  Portfolio Value:      ${portfolio_value:,.2f}")
    print(f"  Total Value:          ${available_balance + portfolio_value:,.2f}")
    print(f"  Last Updated:         {updated_time_et}")
    
    portfolio_summary = {
        'available_balance': available_balance,
        'portfolio_value': portfolio_value,
        'total_value': available_balance + portfolio_value
    }
else:
    print(f"✗ Error: {balance_response.status_code}")
    print(f"Response: {balance_response.text}")
    portfolio_summary = None

# === 2. Get Active Orders ===
print(f"\n{'=' * 70}")
print("ACTIVE ORDERS (Resting)")
print("=" * 70)

orders_response = make_authenticated_request(
    '/trade-api/v2/portfolio/orders',
    params={'status': 'resting', 'limit': 100}
)

if orders_response.status_code == 200:
    orders_data = orders_response.json()
    active_orders = orders_data.get('orders', [])
    
    print(f"\n  Total Active Orders: {len(active_orders)}\n")
    
    if active_orders:
        print(f"  {'Order ID':<20} {'Market':<25} {'Side':<5} {'Action':<6} {'Qty':<6} {'Price':<7} {'Status':<10}")
        print(f"  {'-' * 100}")
        
        for order in active_orders:
            order_id = order.get('order_id', 'N/A')[:18]
            ticker = order.get('ticker', 'N/A')[:23]
            side = order.get('side', 'N/A').upper()
            action = order.get('action', 'N/A')
            qty = order.get('remaining_count', order.get('yes_price', 0))
            price = order.get('yes_price', 0) / 100 if side == 'YES' else order.get('no_price', 0) / 100
            status = order.get('status', 'N/A')
            
            print(f"  {order_id:<20} {ticker:<25} {side:<5} {action:<6} {qty:<6} ${price:<6.2f} {status:<10}")
        
        # Calculate total exposure
        total_yes_exposure = sum(
            (o.get('remaining_count', 0) * o.get('yes_price', 0)) / 100 
            for o in active_orders if o.get('side') == 'yes'
        )
        total_no_exposure = sum(
            (o.get('remaining_count', 0) * o.get('no_price', 0)) / 100 
            for o in active_orders if o.get('side') == 'no'
        )
        
        print(f"\n  Total YES exposure: ${total_yes_exposure:,.2f}")
        print(f"  Total NO exposure:  ${total_no_exposure:,.2f}")
        print(f"  Total exposure:     ${total_yes_exposure + total_no_exposure:,.2f}")
    else:
        print("  No active orders found.")
    
    active_orders_summary = active_orders
else:
    print(f"✗ Error: {orders_response.status_code}")
    print(f"Response: {orders_response.text}")
    active_orders_summary = []

print(f"\n{'=' * 70}\n")

# Store results for further analysis
account_info = {
    'balance': portfolio_summary,
    'active_orders': active_orders_summary
}

account_info

PORTFOLIO BALANCE

  Available Balance:    $4,950.00
  Portfolio Value:      $0.00
  Total Value:          $4,950.00
  Last Updated:         2026-01-31 22:27:08 EST

ACTIVE ORDERS (Resting)

  Total Active Orders: 0

  No active orders found.




{'balance': {'available_balance': 4950.0,
  'portfolio_value': 0.0,
  'total_value': 4950.0},
 'active_orders': []}

In [10]:
import requests

# Fetch all series with the "15 min" tag (crypto 15-minute markets)
url = "https://api.elections.kalshi.com/trade-api/v2/series"
params = {
    "tags": "15 min",  # Filter for 15-minute markets
    "limit": 200
}

response = requests.get(url, params=params)
series_data = response.json()

# Extract series tickers for 15-minute crypto markets
crypto_15min_series = series_data.get('series', [])
print(f"Found {len(crypto_15min_series)} series with '15 min' tag")

# Display first few series to see what we have
for series in crypto_15min_series[:5]:
    print(f"  - {series['ticker']}: {series['title']}")

# Now fetch all markets for these series
all_markets = []
for series in crypto_15min_series:
    series_ticker = series['ticker']
    
    # Fetch markets for this series
    markets_url = "https://api.elections.kalshi.com/trade-api/v2/markets"
    market_params = {
        "series_ticker": series_ticker,
        "status": "open",  # Only get currently open markets
        "limit": 1000
    }
    
    markets_response = requests.get(markets_url, params=market_params)
    markets_data = markets_response.json()
    
    markets = markets_data.get('markets', [])
    all_markets.extend(markets)
    
    if markets:
        print(f"\nFetched {len(markets)} open markets for {series_ticker}")

print(f"\n=== Total: {len(all_markets)} open crypto 15-minute markets ===")

# Store in a variable for further analysis
crypto_15min_markets = all_markets

# Display sample market data
if crypto_15min_markets:
    print("\nSample market:")
    sample = crypto_15min_markets[0]
    print(f"  Ticker: {sample.get('ticker')}")
    print(f"  Title: {sample.get('title')}")
    print(f"  Yes Price: ${sample.get('yes_bid', 0)/100:.2f}")
    print(f"  Volume: {sample.get('volume', 0)}")
    print(f"  Close Time: {sample.get('close_time')}")

crypto_15min_markets

Found 3 series with '15 min' tag
  - KXETH15M: ETH 15M price up down
  - KXBTC15M: Bitcoin price up down
  - KXSOL15M: Solana 15 minutes

Fetched 1 open markets for KXETH15M

Fetched 1 open markets for KXBTC15M

Fetched 1 open markets for KXSOL15M

=== Total: 3 open crypto 15-minute markets ===

Sample market:
  Ticker: KXETH15M-26JAN311745-45
  Title: ETH price up in next 15 mins?
  Yes Price: $0.67
  Volume: 1106
  Close Time: 2026-01-31T22:45:00Z


[{'can_close_early': True,
  'close_time': '2026-01-31T22:45:00Z',
  'created_time': '2026-01-31T01:01:39.986147Z',
  'event_ticker': 'KXETH15M-26JAN311745',
  'expected_expiration_time': '2026-01-31T22:50:00Z',
  'expiration_time': '2026-02-07T22:45:00Z',
  'expiration_value': '',
  'floor_strike': 2414.15,
  'last_price': 69,
  'last_price_dollars': '0.6900',
  'latest_expiration_time': '2026-02-07T22:45:00Z',
  'liquidity': 4625693,
  'liquidity_dollars': '46256.9300',
  'market_type': 'binary',
  'no_ask': 33,
  'no_ask_dollars': '0.3300',
  'no_bid': 29,
  'no_bid_dollars': '0.2900',
  'no_sub_title': 'Price to beat: TBD',
  'notional_value': 100,
  'notional_value_dollars': '1.0000',
  'open_interest': 736,
  'open_interest_fp': '736.00',
  'open_time': '2026-01-31T22:30:00Z',
  'previous_price': 0,
  'previous_price_dollars': '0.0000',
  'previous_yes_ask': 0,
  'previous_yes_ask_dollars': '0.0000',
  'previous_yes_bid': 0,
  'previous_yes_bid_dollars': '0.0000',
  'price_level_

In [34]:
import requests
import time
from datetime import datetime, timedelta, timezone
try:
    from zoneinfo import ZoneInfo
    ET_TZ = ZoneInfo('America/New_York')
except ImportError:
    # Fallback for older Python versions
    from datetime import timezone as tz
    ET_TZ = tz(timedelta(hours=-5))  # EST approximation

def add_et_times(market):
    """Add Eastern Time versions of all time fields to a market dict."""
    time_fields = ['close_time', 'open_time', 'created_time', 'updated_time', 
                   'expiration_time', 'expected_expiration_time', 'latest_expiration_time']
    
    for field in time_fields:
        if field in market and market[field]:
            try:
                # Parse UTC time
                utc_time_str = market[field]
                if isinstance(utc_time_str, str):
                    dt_utc = datetime.fromisoformat(utc_time_str.replace('Z', '+00:00'))
                    # Convert to Eastern Time
                    dt_et = dt_utc.astimezone(ET_TZ)
                    # Format as clean readable string
                    market[f"{field}_et"] = dt_et.strftime('%Y-%m-%d %H:%M:%S %Z')
            except Exception:
                pass  # Skip if conversion fails
    
    return market

# Calculate Unix timestamps for the past month
now = datetime.now(timezone.utc)
one_month_ago = now - timedelta(days=30)

max_close_ts = int(now.timestamp())
min_close_ts = int(one_month_ago.timestamp())

print(f"Fetching Bitcoin 15-min settled markets from the past month:")
print(f"  Start: {one_month_ago.strftime('%Y-%m-%d %H:%M:%S UTC')}")
print(f"  End:   {now.strftime('%Y-%m-%d %H:%M:%S UTC')}")

# The Bitcoin 15-minute series ticker
btc_series_ticker = "KXNYCSNOWM" 

# Fetch all SETTLED markets for Bitcoin 15-minute series in the past month
base_url = "https://api.elections.kalshi.com/trade-api/v2/markets"

all_btc_markets = []
cursor = None
page = 0

while True:
    params = {
        "series_ticker": btc_series_ticker,
        "status": "settled",
        "min_close_ts": min_close_ts,
        "max_close_ts": max_close_ts,
        "limit": 1000
    }
    
    if cursor:
        params["cursor"] = cursor
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 429:
        print("Rate limited, waiting 2 seconds...")
        time.sleep(2)
        continue
    
    if response.status_code != 200:
        print(f"Error: Status {response.status_code}")
        print(f"Response: {response.text}")
        break
    
    data = response.json()
    markets = data.get('markets', [])
    
    # Add Eastern Time fields to each market
    markets_with_et = [add_et_times(m) for m in markets]
    all_btc_markets.extend(markets_with_et)
    
    page += 1
    if markets:
        print(f"Page {page}: Fetched {len(markets)} markets (total: {len(all_btc_markets)})")
    
    cursor = data.get('cursor')
    
    if not cursor:
        break

print(f"\n=== Total: {len(all_btc_markets)} settled BTC 15-min markets ===")

# Store in variable for analysis
btc_15min_historical = all_btc_markets

if btc_15min_historical:
    print("\nMarket Statistics:")
    total_volume = sum(m.get('volume', 0) for m in btc_15min_historical)
    avg_volume = total_volume / len(btc_15min_historical)
    
    print(f"  Total markets: {len(btc_15min_historical)}")
    print(f"  Total volume: {total_volume:,}")
    print(f"  Average volume per market: {avg_volume:.2f}")
    
    # Count results
    yes_count = sum(1 for m in btc_15min_historical if m.get('result') == 'yes')
    no_count = sum(1 for m in btc_15min_historical if m.get('result') == 'no')
    
    print(f"\nOutcomes:")
    print(f"  Yes (price went up): {yes_count} ({yes_count/len(btc_15min_historical)*100:.1f}%)")
    print(f"  No (price went down): {no_count} ({no_count/len(btc_15min_historical)*100:.1f}%)")
    
    print("\nSample markets (first 3) - showing both UTC and ET times:")
    for i, market in enumerate(btc_15min_historical[:3], 1):
        print(f"\n  {i}. {market.get('ticker')}")
        print(f"     Title: {market.get('title')}")
        print(f"     Close time (UTC): {market.get('close_time')}")
        print(f"     Close time (ET):  {market.get('close_time_et')}")
        print(f"     Volume: {market.get('volume', 0)}")
        print(f"     Result: {market.get('result')}")
        print(f"     Final price: ${market.get('last_price', 0)/100:.2f}")
    
    print("\nNote: All markets now have _et suffix fields in format 'YYYY-MM-DD HH:MM:SS EST/EDT'")
    print("      (e.g., close_time_et, open_time_et, created_time_et, etc.)")
else:
    print("\n⚠ No settled markets found in the past 30 days.")
    print("The series might be very new. Try:")
    print("  - Increasing the time window (e.g., 60 or 90 days)")
    print("  - Checking when the series was created")

btc_15min_historical

Fetching Bitcoin 15-min settled markets from the past month:
  Start: 2026-01-02 08:13:34 UTC
  End:   2026-02-01 08:13:34 UTC
Page 1: Fetched 1 markets (total: 1)

=== Total: 1 settled BTC 15-min markets ===

Market Statistics:
  Total markets: 1
  Total volume: 2,491
  Average volume per market: 2491.00

Outcomes:
  Yes (price went up): 1 (100.0%)
  No (price went down): 0 (0.0%)

Sample markets (first 3) - showing both UTC and ET times:

  1. KXNYCSNOWM-26JAN-0.1
     Title: Snow in New York City in Jan 2026?
     Close time (UTC): 2026-01-07T15:00:00Z
     Close time (ET):  2026-01-07 10:00:00 EST
     Volume: 2491
     Result: yes
     Final price: $0.99

Note: All markets now have _et suffix fields in format 'YYYY-MM-DD HH:MM:SS EST/EDT'
      (e.g., close_time_et, open_time_et, created_time_et, etc.)


[{'can_close_early': True,
  'close_time': '2026-01-07T15:00:00Z',
  'created_time': '2025-12-31T21:43:22.690736Z',
  'early_close_condition': 'This market will close and expire early if the event occurs.',
  'event_ticker': 'KXNYCSNOWM-26JAN',
  'expected_expiration_time': '2026-02-02T04:59:59Z',
  'expiration_time': '2026-02-09T04:59:59Z',
  'expiration_value': '',
  'floor_strike': 0.1,
  'last_price': 99,
  'last_price_dollars': '0.9900',
  'latest_expiration_time': '2026-02-09T04:59:59Z',
  'liquidity': 0,
  'liquidity_dollars': '0.0000',
  'market_type': 'binary',
  'no_ask': 100,
  'no_ask_dollars': '1.0000',
  'no_bid': 0,
  'no_bid_dollars': '0.0000',
  'no_sub_title': 'Above 0.1 inches',
  'notional_value': 100,
  'notional_value_dollars': '1.0000',
  'open_interest': 2490,
  'open_interest_fp': '2490.00',
  'open_time': '2026-01-01T00:43:14Z',
  'previous_price': 99,
  'previous_price_dollars': '0.9900',
  'previous_yes_ask': 100,
  'previous_yes_ask_dollars': '1.0000',
  'p

In [21]:
btc_15min_historical[-1]

{'can_close_early': True,
 'close_time': '2026-01-01T23:00:00Z',
 'created_time': '2025-12-31T23:49:48.168266Z',
 'event_ticker': 'KXBTC15M-26JAN011800',
 'expected_expiration_time': '2026-01-01T23:05:00Z',
 'expiration_time': '2026-01-08T23:00:00Z',
 'expiration_value': '',
 'floor_strike': 88456.49,
 'last_price': 99,
 'last_price_dollars': '0.9900',
 'latest_expiration_time': '2026-01-08T23:00:00Z',
 'liquidity': 0,
 'liquidity_dollars': '0.0000',
 'market_type': 'binary',
 'no_ask': 100,
 'no_ask_dollars': '1.0000',
 'no_bid': 0,
 'no_bid_dollars': '0.0000',
 'no_sub_title': 'Price to beat: TBD',
 'notional_value': 100,
 'notional_value_dollars': '1.0000',
 'open_interest': 4458,
 'open_interest_fp': '4458.00',
 'open_time': '2026-01-01T22:45:00Z',
 'previous_price': 99,
 'previous_price_dollars': '0.9900',
 'previous_yes_ask': 100,
 'previous_yes_ask_dollars': '1.0000',
 'previous_yes_bid': 0,
 'previous_yes_bid_dollars': '0.0000',
 'price_level_structure': 'linear_cent',
 'price_

In [25]:
import requests
from datetime import datetime, timedelta, timezone

try:
    from zoneinfo import ZoneInfo
    ET_TZ = ZoneInfo('America/New_York')
except ImportError:
    # Fallback for older Python versions
    ET_TZ = timezone(timedelta(hours=-5))

# Get the last market from historical data
last_market = btc_15min_historical[-1]
ticker = last_market['ticker']
series_ticker = "KXBTC15M"

print(f"=== Analyzing Market: {ticker} ===")
print(f"Title: {last_market['title']}")
print(f"Result: {last_market['result']}")
print(f"Close time: {last_market.get('close_time_et', last_market['close_time'])}")
print(f"\n{'='*60}\n")

# Parse timestamps for the market's lifetime
open_time_str = last_market['open_time']
close_time_str = last_market['close_time']

open_dt = datetime.fromisoformat(open_time_str.replace('Z', '+00:00'))
close_dt = datetime.fromisoformat(close_time_str.replace('Z', '+00:00'))

start_ts = int(open_dt.timestamp())
end_ts = int(close_dt.timestamp())

print(f"Market was active from:")
print(f"  Open:  {open_dt.astimezone(ET_TZ).strftime('%Y-%m-%d %H:%M:%S %Z')}")
print(f"  Close: {close_dt.astimezone(ET_TZ).strftime('%Y-%m-%d %H:%M:%S %Z')}")
print(f"  Duration: {(end_ts - start_ts) / 60:.1f} minutes")

# === 1. Fetch Historical Candlestick Data (Price Evolution) ===
print(f"\n{'='*60}")
print("FETCHING HISTORICAL PRICE DATA (Candlesticks)")
print(f"{'='*60}\n")

candlesticks_url = f"https://api.elections.kalshi.com/trade-api/v2/series/{series_ticker}/markets/{ticker}/candlesticks"
params = {
    "start_ts": start_ts,
    "end_ts": end_ts,
    "period_interval": 1,  # 1-minute intervals for detailed view
    "include_latest_before_start": True
}

candlesticks_response = requests.get(candlesticks_url, params=params)

if candlesticks_response.status_code == 200:
    candlesticks_data = candlesticks_response.json()
    candlesticks = candlesticks_data.get('candlesticks', [])
    
    print(f"✓ Fetched {len(candlesticks)} candlesticks (1-minute intervals)")
    
    if candlesticks:
        first_ts = datetime.fromtimestamp(candlesticks[0]['end_period_ts'], tz=timezone.utc).astimezone(ET_TZ)
        last_ts = datetime.fromtimestamp(candlesticks[-1]['end_period_ts'], tz=timezone.utc).astimezone(ET_TZ)
        
        print("\nPrice Evolution Summary:")
        print(f"  First candlestick: {first_ts.strftime('%H:%M:%S %Z')}")
        print(f"  Last candlestick:  {last_ts.strftime('%H:%M:%S %Z')}")
        
        # Show sample of price movement (every 3 minutes) - YES and NO prices
        print("\nYES Token Price Evolution (every 3 min):")
        print(f"  {'Time (ET)':<12} {'Yes Bid':<9} {'Yes Ask':<9} {'Last Trade':<11} {'Volume':<8}")
        print(f"  {'-'*60}")
        
        for i, candle in enumerate(candlesticks):
            if i % 3 == 0 or i == len(candlesticks) - 1:  # Every 3 minutes + last
                ts = datetime.fromtimestamp(candle['end_period_ts'], tz=timezone.utc).astimezone(ET_TZ)
                time_str = ts.strftime('%H:%M:%S')
                
                yes_bid = candle.get('yes_bid', {}).get('close', 0) if candle.get('yes_bid') else 0
                yes_ask = candle.get('yes_ask', {}).get('close', 0) if candle.get('yes_ask') else 0
                last_price = candle.get('price', {}).get('close', 0) if candle.get('price') else 0
                volume = candle.get('volume', 0)
                
                print(f"  {time_str:<12} ${yes_bid/100:<8.2f} ${yes_ask/100:<8.2f} ${last_price/100:<10.2f} {volume:<8}")
        
        print("\nNO Token Price Evolution (every 3 min):")
        print(f"  {'Time (ET)':<12} {'No Bid':<9} {'No Ask':<9} {'Implied':<11} {'Volume':<8}")
        print(f"  {'-'*60}")
        
        for i, candle in enumerate(candlesticks):
            if i % 3 == 0 or i == len(candlesticks) - 1:  # Every 3 minutes + last
                ts = datetime.fromtimestamp(candle['end_period_ts'], tz=timezone.utc).astimezone(ET_TZ)
                time_str = ts.strftime('%H:%M:%S')
                
                no_bid = candle.get('no_bid', {}).get('close', 0) if candle.get('no_bid') else 0
                no_ask = candle.get('no_ask', {}).get('close', 0) if candle.get('no_ask') else 0
                last_price = candle.get('price', {}).get('close', 0) if candle.get('price') else 0
                # No implied price is 100 - Yes price
                no_implied = 100 - last_price if last_price else 0
                volume = candle.get('volume', 0)
                
                print(f"  {time_str:<12} ${no_bid/100:<8.2f} ${no_ask/100:<8.2f} ${no_implied/100:<10.2f} {volume:<8}")
else:
    print(f"✗ Error fetching candlesticks: {candlesticks_response.status_code}")
    print(f"Response: {candlesticks_response.text}")
    candlesticks = []

# === 2. Fetch Historical Trades ===
print(f"\n{'='*60}")
print("FETCHING HISTORICAL TRADES")
print(f"{'='*60}\n")

trades_url = "https://api.elections.kalshi.com/trade-api/v2/markets/trades"
all_trades = []
cursor = None

while True:
    trade_params = {
        "ticker": ticker,
        "min_ts": start_ts,
        "max_ts": end_ts,
        "limit": 1000
    }
    
    if cursor:
        trade_params["cursor"] = cursor
    
    trades_response = requests.get(trades_url, params=trade_params)
    
    if trades_response.status_code != 200:
        print(f"✗ Error fetching trades: {trades_response.status_code}")
        break
    
    trades_data = trades_response.json()
    trades = trades_data.get('trades', [])
    all_trades.extend(trades)
    
    cursor = trades_data.get('cursor')
    if not cursor:
        break

print(f"✓ Fetched {len(all_trades)} historical trades")

if all_trades:
    # Calculate trade statistics
    total_volume = sum(t.get('count', 0) for t in all_trades)
    yes_trades = [t for t in all_trades if t.get('taker_side') == 'yes']
    no_trades = [t for t in all_trades if t.get('taker_side') == 'no']
    
    print(f"\nTrade Statistics:")
    print(f"  Total volume: {total_volume:,} contracts")
    print(f"  Yes-side trades: {len(yes_trades)} ({len(yes_trades)/len(all_trades)*100:.1f}%)")
    print(f"  No-side trades:  {len(no_trades)} ({len(no_trades)/len(all_trades)*100:.1f}%)")
    
    if yes_trades:
        avg_yes_price = sum(t['yes_price'] for t in yes_trades) / len(yes_trades)
        print(f"  Avg Yes price: ${avg_yes_price/100:.2f}")
    
    if no_trades:
        avg_no_price = sum(t['no_price'] for t in no_trades) / len(no_trades)
        print(f"  Avg No price: ${avg_no_price/100:.2f}")
    
    # Show first and last few trades
    print(f"\nFirst 5 trades:")
    print(f"  {'Time (ET)':<12} {'Side':<4} {'Yes $':<7} {'No $':<7} {'Contracts':<10}")
    print(f"  {'-'*60}")
    
    for trade in all_trades[:5]:
        trade_time = datetime.fromisoformat(trade['created_time'].replace('Z', '+00:00')).astimezone(ET_TZ)
        time_str = trade_time.strftime('%H:%M:%S')
        side = trade['taker_side']
        yes_price = trade['yes_price'] / 100
        no_price = trade['no_price'] / 100
        count = trade['count']
        
        print(f"  {time_str:<12} {side:<4} ${yes_price:<6.2f} ${no_price:<6.2f} {count:<10}")
    
    print(f"\nLast 5 trades:")
    print(f"  {'Time (ET)':<12} {'Side':<4} {'Yes $':<7} {'No $':<7} {'Contracts':<10}")
    print(f"  {'-'*60}")
    
    for trade in all_trades[-5:]:
        trade_time = datetime.fromisoformat(trade['created_time'].replace('Z', '+00:00')).astimezone(ET_TZ)
        time_str = trade_time.strftime('%H:%M:%S')
        side = trade['taker_side']
        yes_price = trade['yes_price'] / 100
        no_price = trade['no_price'] / 100
        count = trade['count']
        
        print(f"  {time_str:<12} {side:<4} ${yes_price:<6.2f} ${no_price:<6.2f} {count:<10}")

# Store data for further analysis
market_historical_data = {
    'market': last_market,
    'candlesticks': candlesticks,
    'trades': all_trades
}

print(f"\n{'='*60}")
print("✓ Data stored in 'market_historical_data' variable")
print(f"{'='*60}\n")

market_historical_data

=== Analyzing Market: KXBTC15M-26JAN011800-00 ===
Title: BTC price up in next 15 mins?
Result: yes
Close time: 2026-01-01 18:00:00 EST


Market was active from:
  Open:  2026-01-01 17:45:00 EST
  Close: 2026-01-01 18:00:00 EST
  Duration: 15.0 minutes

FETCHING HISTORICAL PRICE DATA (Candlesticks)

✓ Fetched 15 candlesticks (1-minute intervals)

Price Evolution Summary:
  First candlestick: 17:46:00 EST
  Last candlestick:  18:00:00 EST

YES Token Price Evolution (every 3 min):
  Time (ET)    Yes Bid   Yes Ask   Last Trade  Volume  
  ------------------------------------------------------------
  17:46:00     $0.67     $0.72     $0.71       65      
  17:49:00     $0.60     $0.65     $0.64       264     
  17:52:00     $0.78     $0.79     $0.79       3488    
  17:55:00     $0.95     $0.96     $0.96       576     
  17:58:00     $0.98     $1.00     $0.99       533     


TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

# Climate Markets Backtesting Strategy

## Strategy Overview
- **Universe**: All climate-related markets from Kalshi
- **Constraints**: 
  - Market duration >= 5 days (120 hours)
  - Liquidity >= $100,000
- **Entry**: 12 hours before market expiration
- **Position Sizing**:
  - Daily budget: $10,000 split across all qualifying events
  - Per event: 70% on YES favorite, 30% split across NO positions
- **Initial Capital**: $100,000
- **Timeframe**: Past year

## Performance Metrics
- Day-by-day equity curve
- Cumulative returns and Sharpe ratio
- Maximum drawdown analysis
- Separate analysis for top 5% liquidity markets

In [28]:
# ============================================================================
# STEP 1: Pull All Historical Climate Markets
# ============================================================================

import requests
import time
from datetime import datetime, timedelta, timezone
import numpy as np
import pandas as pd
try:
    from zoneinfo import ZoneInfo
    ET_TZ = ZoneInfo('America/New_York')
except ImportError:
    ET_TZ = timezone(timedelta(hours=-5))

print("="*80)
print("FETCHING HISTORICAL CLIMATE MARKETS")
print("="*80)

# Calculate time window (past year)
now = datetime.now(timezone.utc)
one_year_ago = now - timedelta(days=365)
min_close_ts = int(one_year_ago.timestamp())
max_close_ts = int(now.timestamp())

print(f"\nTime window:")
print(f"  From: {one_year_ago.strftime('%Y-%m-%d %H:%M:%S UTC')}")
print(f"  To:   {now.strftime('%Y-%m-%d %H:%M:%S UTC')}")

# First, search for climate-related series
print("\n" + "-"*80)
print("Searching for climate series...")
print("-"*80)

base_url = "https://api.elections.kalshi.com/trade-api/v2"

# Try different search approaches for climate markets
search_queries = ["climate", "weather", "temperature", "emissions", "carbon"]
all_series = []

for query in search_queries:
    try:
        response = requests.get(f"{base_url}/series", params={"limit": 200})
        if response.status_code == 200:
            series_data = response.json().get('series', [])
            # Filter series that match climate-related keywords
            climate_series = [
                s for s in series_data 
                if query.lower() in s.get('title', '').lower() 
                or query.lower() in s.get('category', '').lower()
                or any(query.lower() in tag.lower() for tag in s.get('tags', []))
            ]
            all_series.extend(climate_series)
            if climate_series:
                print(f"  Found {len(climate_series)} series matching '{query}'")
        time.sleep(0.5)  # Rate limiting
    except Exception as e:
        print(f"  Error searching for '{query}': {e}")

# Deduplicate series by ticker
unique_series = {s['ticker']: s for s in all_series}.values()
print(f"\n✓ Total unique climate-related series: {len(unique_series)}")

if len(unique_series) == 0:
    print("\n⚠ WARNING: No climate series found with keyword search.")
    print("Fetching ALL series to manually inspect...")
    
    all_response = requests.get(f"{base_url}/series", params={"limit": 1000})
    if all_response.status_code == 200:
        all_series_data = all_response.json().get('series', [])
        print(f"\nTotal series available: {len(all_series_data)}")
        print("\nSample series (first 20):")
        for s in all_series_data[:20]:
            print(f"  - {s['ticker']}: {s['title']} (Category: {s.get('category', 'N/A')})")

# Display found series
if unique_series:
    print("\nClimate series found:")
    for s in unique_series:
        print(f"  - {s['ticker']}: {s['title']}")
        print(f"    Category: {s.get('category', 'N/A')}, Tags: {s.get('tags', [])}")

# Store series tickers for market fetching
climate_series_tickers = [s['ticker'] for s in unique_series]
print(f"\n✓ Will fetch markets for {len(climate_series_tickers)} climate series")

climate_series_list = list(unique_series)

FETCHING HISTORICAL CLIMATE MARKETS

Time window:
  From: 2025-02-01 08:06:36 UTC
  To:   2026-02-01 08:06:36 UTC

--------------------------------------------------------------------------------
Searching for climate series...
--------------------------------------------------------------------------------
  Error searching for 'climate': 'NoneType' object is not iterable
  Error searching for 'weather': 'NoneType' object is not iterable
  Error searching for 'temperature': 'NoneType' object is not iterable
  Error searching for 'emissions': 'NoneType' object is not iterable
  Error searching for 'carbon': 'NoneType' object is not iterable

✓ Total unique climate-related series: 0

⚠ WARNING: No climate series found with keyword search.
Fetching ALL series to manually inspect...

Total series available: 8216

Sample series (first 20):
  - KXUFAWEST: Ultimate Frisbee Association East Division Winner (Category: Sports)
  - KXSTARTIND: iNDIANAPOLIS WEEK 1 QB (Category: Sports)
  - DEBATE

In [32]:
unique_series

dict_values([])

In [33]:
# ============================================================================
# STEP 2: Fetch All Markets for Climate Series
# ============================================================================

print("\n" + "="*80)
print("FETCHING MARKETS FOR CLIMATE SERIES")
print("="*80)

# If no climate series found, fall back to manually specifying some categories
if len(climate_series_tickers) == 0:
    print("\n⚠ No climate series found. Trying category-based search...")
    
    # Try fetching by category
    categories_to_try = ["Science", "Environment", "Weather"]
    
    for category in categories_to_try:
        try:
            response = requests.get(f"{base_url}/series", params={"category": category, "limit": 200})
            if response.status_code == 200:
                series = response.json().get('series', [])
                print(f"\n  Category '{category}': Found {len(series)} series")
                if series:
                    for s in series[:5]:
                        print(f"    - {s['ticker']}: {s['title']}")
                all_series.extend(series)
            time.sleep(0.5)
        except Exception as e:
            print(f"  Error fetching category '{category}': {e}")
    
    unique_series = {s['ticker']: s for s in all_series}.values()
    climate_series_tickers = [s['ticker'] for s in unique_series]
    climate_series_list = list(unique_series)

# Fetch all markets for these series
all_markets = []

print(f"\nFetching settled markets for {len(climate_series_tickers)} series...")
print("(This may take a few minutes...)\n")

for i, series_ticker in enumerate(climate_series_tickers, 1):
    cursor = None
    series_markets = []
    
    while True:
        params = {
            "series_ticker": series_ticker,
            "status": "settled",  # Only settled markets for backtesting
            "min_close_ts": min_close_ts,
            "max_close_ts": max_close_ts,
            "limit": 1000
        }
        
        if cursor:
            params["cursor"] = cursor
        
        try:
            response = requests.get(f"{base_url}/markets", params=params)
            
            if response.status_code == 429:
                print(f"  Rate limited, waiting 2 seconds...")
                time.sleep(2)
                continue
            
            if response.status_code != 200:
                print(f"  ✗ Error for {series_ticker}: Status {response.status_code}")
                break
            
            data = response.json()
            markets = data.get('markets', [])
            series_markets.extend(markets)
            
            cursor = data.get('cursor')
            if not cursor:
                break
            
            time.sleep(0.3)  # Rate limiting
            
        except Exception as e:
            print(f"  ✗ Error fetching {series_ticker}: {e}")
            break
    
    if series_markets:
        all_markets.extend(series_markets)
        print(f"  [{i}/{len(climate_series_tickers)}] {series_ticker}: {len(series_markets)} markets")
    
    # Progress update every 5 series
    if i % 5 == 0:
        print(f"  Progress: {i}/{len(climate_series_tickers)} series processed, {len(all_markets)} total markets")

print(f"\n{'='*80}")
print(f"✓ TOTAL MARKETS FETCHED: {len(all_markets)}")
print(f"{'='*80}")

# Store raw markets
climate_markets_raw = all_markets


FETCHING MARKETS FOR CLIMATE SERIES

⚠ No climate series found. Trying category-based search...
  Error fetching category 'Science': object of type 'NoneType' has no len()
  Error fetching category 'Environment': object of type 'NoneType' has no len()
  Error fetching category 'Weather': object of type 'NoneType' has no len()

Fetching settled markets for 0 series...
(This may take a few minutes...)


✓ TOTAL MARKETS FETCHED: 0


In [ ]:
# ============================================================================
# STEP 3: Filter Markets by Duration and Liquidity Constraints
# ============================================================================

print("\n" + "="*80)
print("APPLYING FILTERS: DURATION >= 5 DAYS, LIQUIDITY >= $100,000")
print("="*80)

# Target thresholds
MIN_DURATION_HOURS = 5 * 24  # 5 days = 120 hours
MIN_LIQUIDITY_DOLLARS = 100000  # $100,000

# Apply filters
filtered_markets = []
filter_stats = {
    'total': len(climate_markets_raw),
    'duration_pass': 0,
    'liquidity_pass': 0,
    'both_pass': 0,
    'duration_fail': 0,
    'liquidity_fail': 0
}

for market in climate_markets_raw:
    # Parse timestamps
    try:
        open_time = datetime.fromisoformat(market['open_time'].replace('Z', '+00:00'))
        close_time = datetime.fromisoformat(market['close_time'].replace('Z', '+00:00'))
        duration_hours = (close_time - open_time).total_seconds() / 3600
        
        liquidity_dollars = float(market.get('liquidity_dollars', '0'))
        
        # Track filter pass rates
        duration_ok = duration_hours >= MIN_DURATION_HOURS
        liquidity_ok = liquidity_dollars >= MIN_LIQUIDITY_DOLLARS
        
        if duration_ok:
            filter_stats['duration_pass'] += 1
        else:
            filter_stats['duration_fail'] += 1
        
        if liquidity_ok:
            filter_stats['liquidity_pass'] += 1
        else:
            filter_stats['liquidity_fail'] += 1
        
        # Both filters must pass
        if duration_ok and liquidity_ok:
            market['duration_hours'] = duration_hours
            market['liquidity_dollars_float'] = liquidity_dollars
            filtered_markets.append(market)
            filter_stats['both_pass'] += 1
            
    except Exception as e:
        print(f"  Error processing market {market.get('ticker', 'unknown')}: {e}")
        continue

print(f"\nFilter Results:")
print(f"  Total markets:           {filter_stats['total']}")
print(f"  Duration filter (>= 5d): {filter_stats['duration_pass']} passed, {filter_stats['duration_fail']} failed")
print(f"  Liquidity filter (>=$100k): {filter_stats['liquidity_pass']} passed, {filter_stats['liquidity_fail']} failed")
print(f"  ✓ Both filters passed:   {filter_stats['both_pass']} markets")

# If no markets pass strict filters, try lower thresholds
if filter_stats['both_pass'] == 0:
    print("\n" + "⚠"*40)
    print("⚠ NO MARKETS PASSED STRICT FILTERS")
    print("⚠ Lowering thresholds to find available data...")
    print("⚠"*40)
    
    # Try progressively lower thresholds
    thresholds_to_try = [
        (3 * 24, 50000),   # 3 days, $50k
        (2 * 24, 25000),   # 2 days, $25k
        (1 * 24, 10000),   # 1 day, $10k
        (0, 0)             # No filters
    ]
    
    for min_dur, min_liq in thresholds_to_try:
        filtered_markets = []
        for market in climate_markets_raw:
            try:
                open_time = datetime.fromisoformat(market['open_time'].replace('Z', '+00:00'))
                close_time = datetime.fromisoformat(market['close_time'].replace('Z', '+00:00'))
                duration_hours = (close_time - open_time).total_seconds() / 3600
                liquidity_dollars = float(market.get('liquidity_dollars', '0'))
                
                if duration_hours >= min_dur and liquidity_dollars >= min_liq:
                    market['duration_hours'] = duration_hours
                    market['liquidity_dollars_float'] = liquidity_dollars
                    filtered_markets.append(market)
            except:
                continue
        
        if len(filtered_markets) > 0:
            print(f"\n✓ Found {len(filtered_markets)} markets with:")
            print(f"  - Duration >= {min_dur/24:.1f} days")
            print(f"  - Liquidity >= ${min_liq:,}")
            MIN_DURATION_HOURS = min_dur
            MIN_LIQUIDITY_DOLLARS = min_liq
            break
    
    if len(filtered_markets) == 0:
        print("\n⚠ WARNING: No climate markets found even with lowered thresholds!")
        print("Showing sample of available markets:\n")
        for i, m in enumerate(climate_markets_raw[:5], 1):
            try:
                open_time = datetime.fromisoformat(m['open_time'].replace('Z', '+00:00'))
                close_time = datetime.fromisoformat(m['close_time'].replace('Z', '+00:00'))
                duration = (close_time - open_time).total_seconds() / 3600
                liq = float(m.get('liquidity_dollars', '0'))
                print(f"  {i}. {m['ticker']}")
                print(f"     Duration: {duration/24:.2f} days, Liquidity: ${liq:,.2f}")
            except:
                pass

print(f"\n{'='*80}")
print(f"✓ QUALIFIED MARKETS: {len(filtered_markets)}")
print(f"{'='*80}")

# Show sample of qualified markets
if filtered_markets:
    print("\nSample qualified markets (first 5):")
    for i, m in enumerate(filtered_markets[:5], 1):
        print(f"\n  {i}. {m['ticker']}")
        print(f"     Title: {m.get('title', 'N/A')}")
        print(f"     Duration: {m['duration_hours']/24:.2f} days")
        print(f"     Liquidity: ${m['liquidity_dollars_float']:,.2f}")
        print(f"     Open: {m['open_time']}")
        print(f"     Close: {m['close_time']}")
        print(f"     Result: {m.get('result', 'N/A')}")

climate_markets_qualified = filtered_markets

In [ ]:
# ============================================================================
# STEP 4: Group by Event and Calculate Entry Points
# ============================================================================

print("\n" + "="*80)
print("GROUPING MARKETS BY EVENT & CALCULATING ENTRY POINTS")
print("="*80)

# Group markets by event_ticker to identify multi-decision markets
events_dict = {}

for market in climate_markets_qualified:
    event_ticker = market.get('event_ticker', market['ticker'])
    
    if event_ticker not in events_dict:
        events_dict[event_ticker] = []
    
    # Calculate entry point (12 hours before close)
    close_time = datetime.fromisoformat(market['close_time'].replace('Z', '+00:00'))
    entry_time = close_time - timedelta(hours=12)
    
    # Add entry info to market
    market['entry_time'] = entry_time
    market['entry_time_str'] = entry_time.isoformat()
    market['close_time_dt'] = close_time
    
    events_dict[event_ticker].append(market)

print(f"\n✓ Found {len(events_dict)} unique events")

# Classify events by number of markets (outcomes)
event_classification = {
    'single': [],      # 1 market (binary event)
    'multi': []        # 2+ markets (multi-decision event)
}

for event_ticker, markets in events_dict.items():
    if len(markets) == 1:
        event_classification['single'].append(event_ticker)
    else:
        event_classification['multi'].append(event_ticker)

print(f"\nEvent Classification:")
print(f"  Single-outcome events: {len(event_classification['single'])}")
print(f"  Multi-outcome events:  {len(event_classification['multi'])}")

# Focus on multi-decision events for the strategy
multi_decision_events = {k: v for k, v in events_dict.items() if len(v) >= 2}

print(f"\n✓ Multi-decision events for strategy: {len(multi_decision_events)}")

# Show sample multi-decision events
if multi_decision_events:
    print("\nSample multi-decision events (first 3):")
    for i, (event_ticker, markets) in enumerate(list(multi_decision_events.items())[:3], 1):
        print(f"\n  {i}. Event: {event_ticker}")
        print(f"     Number of markets: {len(markets)}")
        print(f"     Entry time: {markets[0]['entry_time'].strftime('%Y-%m-%d %H:%M:%S UTC')}")
        print(f"     Close time: {markets[0]['close_time_dt'].strftime('%Y-%m-%d %H:%M:%S UTC')}")
        print(f"     Markets:")
        for m in markets:
            print(f"       - {m['ticker']}: {m.get('title', 'N/A')[:60]}...")
            print(f"         Yes bid/ask: ${m.get('yes_bid', 0)/100:.2f}/${m.get('yes_ask', 0)/100:.2f}")
            print(f"         Result: {m.get('result', 'N/A')}")

# Also include single binary events for completeness
all_tradeable_events = events_dict.copy()

print(f"\n✓ Total tradeable events (single + multi): {len(all_tradeable_events)}")
print(f"{'='*80}")

In [ ]:
# ============================================================================
# STEP 5: Organize Events by Entry Date for Daily Trading
# ============================================================================

print("\n" + "="*80)
print("ORGANIZING EVENTS BY ENTRY DATE")
print("="*80)

# Organize events by entry date (just the date, not time)
events_by_entry_date = {}

for event_ticker, markets in all_tradeable_events.items():
    # All markets in an event have the same entry time
    entry_time = markets[0]['entry_time']
    entry_date = entry_time.date()
    
    if entry_date not in events_by_entry_date:
        events_by_entry_date[entry_date] = []
    
    events_by_entry_date[entry_date].append({
        'event_ticker': event_ticker,
        'markets': markets,
        'entry_time': entry_time,
        'num_markets': len(markets)
    })

# Sort dates
sorted_dates = sorted(events_by_entry_date.keys())

print(f"\n✓ Trading spans {len(sorted_dates)} unique dates")
print(f"  First entry date: {sorted_dates[0] if sorted_dates else 'N/A'}")
print(f"  Last entry date:  {sorted_dates[-1] if sorted_dates else 'N/A'}")

# Show distribution of events per day
events_per_day = [len(events) for events in events_by_entry_date.values()]
if events_per_day:
    print(f"\nEvents per day statistics:")
    print(f"  Average: {np.mean(events_per_day):.2f}")
    print(f"  Median:  {np.median(events_per_day):.0f}")
    print(f"  Max:     {np.max(events_per_day)}")
    print(f"  Min:     {np.min(events_per_day)}")

# Show sample daily schedule
print(f"\nSample daily schedule (first 5 days):")
for i, date in enumerate(sorted_dates[:5], 1):
    events = events_by_entry_date[date]
    print(f"\n  Day {i}: {date}")
    print(f"  Events to trade: {len(events)}")
    for event in events[:3]:  # Show first 3 events
        print(f"    - {event['event_ticker']}: {event['num_markets']} markets")
    if len(events) > 3:
        print(f"    ... and {len(events) - 3} more events")

print(f"\n{'='*80}")

In [ ]:
# ============================================================================
# STEP 6: Simulate Trading Strategy
# ============================================================================

print("\n" + "="*80)
print("SIMULATING TRADING STRATEGY")
print("="*80)

# Strategy parameters
INITIAL_CAPITAL = 100000  # $100,000
DAILY_BUDGET = 10000      # $10,000 per day
YES_ALLOCATION = 0.70     # 70% on YES favorite
NO_ALLOCATION = 0.30      # 30% split across NOs

# Track all trades
all_trades = []
trade_id = 0

print(f"\nStrategy Parameters:")
print(f"  Initial capital:  ${INITIAL_CAPITAL:,}")
print(f"  Daily budget:     ${DAILY_BUDGET:,}")
print(f"  YES allocation:   {YES_ALLOCATION*100:.0f}%")
print(f"  NO allocation:    {NO_ALLOCATION*100:.0f}% (split across NOs)")

print(f"\n{'-'*80}")
print("Processing trades...")
print("-"*80)

# Process each trading day
for date in sorted_dates:
    events_today = events_by_entry_date[date]
    num_events = len(events_today)
    
    if num_events == 0:
        continue
    
    # Allocate budget across events
    budget_per_event = DAILY_BUDGET / num_events
    
    # Process each event
    for event in events_today:
        event_ticker = event['event_ticker']
        markets = event['markets']
        num_markets = len(markets)
        
        # For multi-decision events: YES on favorite, NO on others
        # For single binary: YES if probability > 50%, else NO
        
        if num_markets >= 2:
            # Multi-decision: identify favorite (highest yes_bid)
            favorite = max(markets, key=lambda m: m.get('yes_bid', 0))
            others = [m for m in markets if m != favorite]
            
            # Position sizing:
            # 70% on YES favorite
            # 30% split across NO on others
            yes_size_dollars = budget_per_event * YES_ALLOCATION
            no_size_dollars = budget_per_event * NO_ALLOCATION
            no_size_per_market = no_size_dollars / len(others) if others else 0
            
            # Trade 1: Buy YES on favorite
            yes_price = favorite.get('yes_ask', favorite.get('yes_bid', 50))  # Use ask price
            yes_contracts = (yes_size_dollars * 100) / yes_price if yes_price > 0 else 0
            
            # Calculate P&L based on result
            if favorite.get('result') == 'yes':
                yes_pnl = yes_contracts * (100 - yes_price) / 100  # Win: get $1 per contract, paid yes_price
            elif favorite.get('result') == 'no':
                yes_pnl = -yes_size_dollars  # Loss: lose entire investment
            elif favorite.get('result') == 'void':
                yes_pnl = 0  # Refund
            else:
                yes_pnl = 0  # Unknown result
            
            trade_id += 1
            all_trades.append({
                'trade_id': trade_id,
                'event_ticker': event_ticker,
                'ticker': favorite['ticker'],
                'entry_date': date,
                'entry_time': event['entry_time'],
                'close_time': favorite['close_time_dt'],
                'side': 'YES',
                'price': yes_price / 100,
                'contracts': yes_contracts,
                'investment': yes_size_dollars,
                'result': favorite.get('result', 'unknown'),
                'pnl': yes_pnl,
                'market_type': 'multi_favorite',
                'liquidity': favorite.get('liquidity_dollars_float', 0),
                'duration_days': favorite.get('duration_hours', 0) / 24
            })
            
            # Trades 2+: Buy NO on all others
            for other in others:
                no_price = other.get('no_ask', other.get('no_bid', 50))
                no_contracts = (no_size_per_market * 100) / no_price if no_price > 0 else 0
                
                # Calculate P&L
                if other.get('result') == 'no':
                    no_pnl = no_contracts * (100 - no_price) / 100  # Win
                elif other.get('result') == 'yes':
                    no_pnl = -no_size_per_market  # Loss
                elif other.get('result') == 'void':
                    no_pnl = 0  # Refund
                else:
                    no_pnl = 0
                
                trade_id += 1
                all_trades.append({
                    'trade_id': trade_id,
                    'event_ticker': event_ticker,
                    'ticker': other['ticker'],
                    'entry_date': date,
                    'entry_time': event['entry_time'],
                    'close_time': other['close_time_dt'],
                    'side': 'NO',
                    'price': no_price / 100,
                    'contracts': no_contracts,
                    'investment': no_size_per_market,
                    'result': other.get('result', 'unknown'),
                    'pnl': no_pnl,
                    'market_type': 'multi_other',
                    'liquidity': other.get('liquidity_dollars_float', 0),
                    'duration_days': other.get('duration_hours', 0) / 24
                })
        
        else:
            # Single binary market: buy YES if implied prob > 50%, else NO
            market = markets[0]
            yes_bid = market.get('yes_bid', 50)
            
            if yes_bid >= 50:
                # Buy YES
                yes_price = market.get('yes_ask', yes_bid)
                contracts = (budget_per_event * 100) / yes_price if yes_price > 0 else 0
                
                if market.get('result') == 'yes':
                    pnl = contracts * (100 - yes_price) / 100
                elif market.get('result') == 'no':
                    pnl = -budget_per_event
                elif market.get('result') == 'void':
                    pnl = 0
                else:
                    pnl = 0
                
                side = 'YES'
                price = yes_price
            else:
                # Buy NO
                no_price = market.get('no_ask', 100 - yes_bid)
                contracts = (budget_per_event * 100) / no_price if no_price > 0 else 0
                
                if market.get('result') == 'no':
                    pnl = contracts * (100 - no_price) / 100
                elif market.get('result') == 'yes':
                    pnl = -budget_per_event
                elif market.get('result') == 'void':
                    pnl = 0
                else:
                    pnl = 0
                
                side = 'NO'
                price = no_price
            
            trade_id += 1
            all_trades.append({
                'trade_id': trade_id,
                'event_ticker': event_ticker,
                'ticker': market['ticker'],
                'entry_date': date,
                'entry_time': event['entry_time'],
                'close_time': market['close_time_dt'],
                'side': side,
                'price': price / 100,
                'contracts': contracts,
                'investment': budget_per_event,
                'result': market.get('result', 'unknown'),
                'pnl': pnl,
                'market_type': 'binary',
                'liquidity': market.get('liquidity_dollars_float', 0),
                'duration_days': market.get('duration_hours', 0) / 24
            })

print(f"\n✓ Simulation complete!")
print(f"  Total trades executed: {len(all_trades)}")
print(f"  Trading days: {len(sorted_dates)}")
print(f"{'='*80}")

# Convert to DataFrame for easier analysis
trades_df = pd.DataFrame(all_trades)

# Show sample trades
print(f"\nSample trades (first 10):")
if not trades_df.empty:
    print(trades_df[['trade_id', 'ticker', 'side', 'price', 'investment', 'result', 'pnl']].head(10).to_string(index=False))

In [ ]:
# ============================================================================
# STEP 7: Build Day-by-Day Equity Curve
# ============================================================================

print("\n" + "="*80)
print("BUILDING DAILY EQUITY CURVE")
print("="*80)

if trades_df.empty:
    print("\n⚠ WARNING: No trades to analyze!")
else:
    # Build daily P&L by settlement date (close_time)
    # Group trades by the date they close (when P&L is realized)
    trades_df['close_date'] = trades_df['close_time'].dt.date
    
    # Calculate daily P&L
    daily_pnl = trades_df.groupby('close_date')['pnl'].sum().sort_index()
    
    # Build equity curve
    equity_curve = pd.Series(dtype=float)
    equity_curve[daily_pnl.index[0]] = INITIAL_CAPITAL  # Starting capital
    
    # Accumulate daily P&L
    for date in daily_pnl.index:
        prev_date = equity_curve.index[-1]
        prev_equity = equity_curve.iloc[-1]
        
        # Add daily P&L to equity
        new_equity = prev_equity + daily_pnl[date]
        equity_curve[date] = new_equity
    
    # Create DataFrame for easier analysis
    equity_df = pd.DataFrame({
        'date': equity_curve.index,
        'equity': equity_curve.values,
        'daily_pnl': [0] + list(daily_pnl.values)
    })
    
    # Calculate daily returns
    equity_df['daily_return'] = equity_df['equity'].pct_change()
    equity_df['cumulative_return'] = (equity_df['equity'] / INITIAL_CAPITAL - 1) * 100
    
    # Calculate running drawdown
    equity_df['running_max'] = equity_df['equity'].cummax()
    equity_df['drawdown'] = (equity_df['equity'] - equity_df['running_max']) / equity_df['running_max'] * 100
    
    print(f"\n✓ Equity curve built!")
    print(f"  Trading days: {len(equity_df)}")
    print(f"  Start date: {equity_df['date'].iloc[0]}")
    print(f"  End date: {equity_df['date'].iloc[-1]}")
    
    print(f"\nEquity Summary:")
    print(f"  Starting capital: ${INITIAL_CAPITAL:,.2f}")
    print(f"  Ending equity:    ${equity_df['equity'].iloc[-1]:,.2f}")
    print(f"  Total P&L:        ${equity_df['equity'].iloc[-1] - INITIAL_CAPITAL:,.2f}")
    print(f"  Total return:     {equity_df['cumulative_return'].iloc[-1]:.2f}%")
    
    # Show daily equity progression (sample)
    print(f"\nDaily equity (first 10 days):")
    print(equity_df[['date', 'equity', 'daily_pnl', 'cumulative_return', 'drawdown']].head(10).to_string(index=False))
    
    print(f"\nDaily equity (last 10 days):")
    print(equity_df[['date', 'equity', 'daily_pnl', 'cumulative_return', 'drawdown']].tail(10).to_string(index=False))

In [ ]:
# ============================================================================
# STEP 8: Calculate Comprehensive Performance Metrics
# ============================================================================

print("\n" + "="*80)
print("CALCULATING PERFORMANCE METRICS")
print("="*80)

if not trades_df.empty and len(equity_df) > 1:
    # ========== Returns Analysis ==========
    total_return_pct = (equity_df['equity'].iloc[-1] / INITIAL_CAPITAL - 1) * 100
    total_pnl = equity_df['equity'].iloc[-1] - INITIAL_CAPITAL
    
    # Calculate annualized return
    days_trading = (equity_df['date'].iloc[-1] - equity_df['date'].iloc[0]).days
    years_trading = days_trading / 365.25
    cagr = ((equity_df['equity'].iloc[-1] / INITIAL_CAPITAL) ** (1 / years_trading) - 1) * 100 if years_trading > 0 else 0
    
    # ========== Risk Metrics ==========
    # Daily returns for Sharpe calculation (exclude first day with 0 return)
    daily_returns = equity_df['daily_return'].dropna()
    
    if len(daily_returns) > 1:
        avg_daily_return = daily_returns.mean()
        std_daily_return = daily_returns.std()
        
        # Sharpe ratio (annualized, assuming 252 trading days, risk-free rate = 0)
        sharpe_ratio = (avg_daily_return / std_daily_return * np.sqrt(252)) if std_daily_return > 0 else 0
        
        # Sortino ratio (downside deviation)
        downside_returns = daily_returns[daily_returns < 0]
        downside_std = downside_returns.std() if len(downside_returns) > 1 else 0
        sortino_ratio = (avg_daily_return / downside_std * np.sqrt(252)) if downside_std > 0 else 0
    else:
        sharpe_ratio = 0
        sortino_ratio = 0
        std_daily_return = 0
    
    # ========== Drawdown Analysis ==========
    max_drawdown = equity_df['drawdown'].min()  # Most negative value
    max_drawdown_idx = equity_df['drawdown'].idxmin()
    max_drawdown_date = equity_df.loc[max_drawdown_idx, 'date']
    
    # Average drawdown
    avg_drawdown = equity_df['drawdown'][equity_df['drawdown'] < 0].mean()
    
    # Drawdown duration (consecutive days in drawdown)
    in_drawdown = equity_df['drawdown'] < -0.01  # More than 0.01% drawdown
    drawdown_periods = in_drawdown.astype(int).diff().fillna(0)
    
    # ========== Trade Statistics ==========
    total_trades = len(trades_df)
    winning_trades = len(trades_df[trades_df['pnl'] > 0])
    losing_trades = len(trades_df[trades_df['pnl'] < 0])
    breakeven_trades = len(trades_df[trades_df['pnl'] == 0])
    
    win_rate = (winning_trades / total_trades * 100) if total_trades > 0 else 0
    
    avg_win = trades_df[trades_df['pnl'] > 0]['pnl'].mean() if winning_trades > 0 else 0
    avg_loss = trades_df[trades_df['pnl'] < 0]['pnl'].mean() if losing_trades > 0 else 0
    
    profit_factor = (trades_df[trades_df['pnl'] > 0]['pnl'].sum() / 
                     abs(trades_df[trades_df['pnl'] < 0]['pnl'].sum())) if losing_trades > 0 else float('inf')
    
    # Expectancy per trade
    expectancy = trades_df['pnl'].mean()
    
    # ========== Display Results ==========
    print("\n" + "="*80)
    print("OVERALL STRATEGY PERFORMANCE")
    print("="*80)
    
    print(f"\n📊 RETURNS:")
    print(f"  Total Return:        {total_return_pct:>10.2f}%")
    print(f"  Total P&L:           ${total_pnl:>10,.2f}")
    print(f"  CAGR:                {cagr:>10.2f}%")
    print(f"  Trading Period:      {days_trading} days ({years_trading:.2f} years)")
    
    print(f"\n📉 RISK METRICS:")
    print(f"  Sharpe Ratio:        {sharpe_ratio:>10.2f}")
    print(f"  Sortino Ratio:       {sortino_ratio:>10.2f}")
    print(f"  Daily Volatility:    {std_daily_return*100:>10.2f}%")
    print(f"  Max Drawdown:        {max_drawdown:>10.2f}% (on {max_drawdown_date})")
    print(f"  Avg Drawdown:        {avg_drawdown:>10.2f}%")
    
    print(f"\n💰 TRADE STATISTICS:")
    print(f"  Total Trades:        {total_trades:>10,}")
    print(f"  Winning Trades:      {winning_trades:>10,} ({win_rate:.1f}%)")
    print(f"  Losing Trades:       {losing_trades:>10,}")
    print(f"  Breakeven Trades:    {breakeven_trades:>10,}")
    print(f"  Win Rate:            {win_rate:>10.1f}%")
    print(f"  Avg Win:             ${avg_win:>10,.2f}")
    print(f"  Avg Loss:            ${avg_loss:>10,.2f}")
    print(f"  Profit Factor:       {profit_factor if profit_factor != float('inf') else 'N/A':>10}")
    print(f"  Expectancy/Trade:    ${expectancy:>10,.2f}")
    
    print(f"\n💵 CAPITAL EFFICIENCY:")
    total_invested = trades_df['investment'].sum()
    print(f"  Total Invested:      ${total_invested:>10,.2f}")
    print(f"  ROI on Invested:     {(total_pnl / total_invested * 100) if total_invested > 0 else 0:>10.2f}%")
    
    # Store metrics for later use
    performance_metrics = {
        'total_return_pct': total_return_pct,
        'total_pnl': total_pnl,
        'cagr': cagr,
        'sharpe_ratio': sharpe_ratio,
        'sortino_ratio': sortino_ratio,
        'max_drawdown': max_drawdown,
        'avg_drawdown': avg_drawdown,
        'win_rate': win_rate,
        'profit_factor': profit_factor,
        'expectancy': expectancy,
        'total_trades': total_trades,
        'winning_trades': winning_trades,
        'losing_trades': losing_trades
    }

else:
    print("\n⚠ WARNING: Insufficient data for metrics calculation!")
    performance_metrics = {}

print(f"\n{'='*80}")

In [ ]:
# ============================================================================
# STEP 9: Analyze Top 5% Liquidity Markets Separately
# ============================================================================

print("\n" + "="*80)
print("ANALYZING TOP 5% LIQUIDITY MARKETS")
print("="*80)

if not trades_df.empty:
    # Calculate 95th percentile of liquidity
    liquidity_95th_percentile = trades_df['liquidity'].quantile(0.95)
    
    print(f"\nLiquidity threshold (top 5%):")
    print(f"  95th percentile: ${liquidity_95th_percentile:,.2f}")
    
    # Filter to top 5% liquidity trades
    top_5_pct_trades = trades_df[trades_df['liquidity'] >= liquidity_95th_percentile].copy()
    
    print(f"\n✓ Found {len(top_5_pct_trades)} trades in top 5% liquidity")
    print(f"  ({len(top_5_pct_trades) / len(trades_df) * 100:.1f}% of all trades)")
    
    if len(top_5_pct_trades) > 0:
        # Calculate metrics for top 5% markets
        top_5_total_pnl = top_5_pct_trades['pnl'].sum()
        top_5_winning = len(top_5_pct_trades[top_5_pct_trades['pnl'] > 0])
        top_5_losing = len(top_5_pct_trades[top_5_pct_trades['pnl'] < 0])
        top_5_win_rate = (top_5_winning / len(top_5_pct_trades) * 100) if len(top_5_pct_trades) > 0 else 0
        
        top_5_avg_win = top_5_pct_trades[top_5_pct_trades['pnl'] > 0]['pnl'].mean() if top_5_winning > 0 else 0
        top_5_avg_loss = top_5_pct_trades[top_5_pct_trades['pnl'] < 0]['pnl'].mean() if top_5_losing > 0 else 0
        
        top_5_profit_factor = (top_5_pct_trades[top_5_pct_trades['pnl'] > 0]['pnl'].sum() / 
                               abs(top_5_pct_trades[top_5_pct_trades['pnl'] < 0]['pnl'].sum())) if top_5_losing > 0 else float('inf')
        
        # Build equity curve for top 5% markets only
        top_5_daily_pnl = top_5_pct_trades.groupby('close_date')['pnl'].sum().sort_index()
        
        # Construct top 5% equity curve
        top_5_equity_curve = pd.Series(dtype=float)
        top_5_equity_curve[top_5_daily_pnl.index[0]] = INITIAL_CAPITAL
        
        for date in top_5_daily_pnl.index:
            prev_equity = top_5_equity_curve.iloc[-1]
            new_equity = prev_equity + top_5_daily_pnl[date]
            top_5_equity_curve[date] = new_equity
        
        top_5_equity_df = pd.DataFrame({
            'date': top_5_equity_curve.index,
            'equity': top_5_equity_curve.values,
            'daily_pnl': [0] + list(top_5_daily_pnl.values)
        })
        
        top_5_equity_df['daily_return'] = top_5_equity_df['equity'].pct_change()
        top_5_equity_df['running_max'] = top_5_equity_df['equity'].cummax()
        top_5_equity_df['drawdown'] = (top_5_equity_df['equity'] - top_5_equity_df['running_max']) / top_5_equity_df['running_max'] * 100
        
        # Calculate Sharpe ratio for top 5%
        top_5_daily_returns = top_5_equity_df['daily_return'].dropna()
        if len(top_5_daily_returns) > 1:
            top_5_avg_return = top_5_daily_returns.mean()
            top_5_std_return = top_5_daily_returns.std()
            top_5_sharpe = (top_5_avg_return / top_5_std_return * np.sqrt(252)) if top_5_std_return > 0 else 0
        else:
            top_5_sharpe = 0
        
        top_5_max_dd = top_5_equity_df['drawdown'].min()
        top_5_total_return = (top_5_equity_df['equity'].iloc[-1] / INITIAL_CAPITAL - 1) * 100
        
        # Display results
        print("\n" + "="*80)
        print("TOP 5% LIQUIDITY MARKETS PERFORMANCE")
        print("="*80)
        
        print(f"\n📊 RETURNS:")
        print(f"  Total P&L:           ${top_5_total_pnl:>10,.2f}")
        print(f"  Total Return:        {top_5_total_return:>10.2f}%")
        print(f"  Contribution to Total: {(top_5_total_pnl / total_pnl * 100) if total_pnl != 0 else 0:>8.1f}%")
        
        print(f"\n📉 RISK METRICS:")
        print(f"  Sharpe Ratio:        {top_5_sharpe:>10.2f}")
        print(f"  Max Drawdown:        {top_5_max_dd:>10.2f}%")
        
        print(f"\n💰 TRADE STATISTICS:")
        print(f"  Total Trades:        {len(top_5_pct_trades):>10,}")
        print(f"  Win Rate:            {top_5_win_rate:>10.1f}%")
        print(f"  Avg Win:             ${top_5_avg_win:>10,.2f}")
        print(f"  Avg Loss:            ${top_5_avg_loss:>10,.2f}")
        print(f"  Profit Factor:       {top_5_profit_factor if top_5_profit_factor != float('inf') else 'N/A':>10}")
        
        # Comparison with overall strategy
        print(f"\n📈 COMPARISON VS OVERALL:")
        print(f"  Win Rate Diff:       {top_5_win_rate - win_rate:>+10.1f} pp")
        print(f"  Sharpe Diff:         {top_5_sharpe - sharpe_ratio:>+10.2f}")
        print(f"  Max DD Diff:         {top_5_max_dd - max_drawdown:>+10.2f} pp")
        
        # Store top 5% metrics
        top_5_metrics = {
            'total_pnl': top_5_total_pnl,
            'total_return': top_5_total_return,
            'sharpe_ratio': top_5_sharpe,
            'max_drawdown': top_5_max_dd,
            'win_rate': top_5_win_rate,
            'profit_factor': top_5_profit_factor,
            'num_trades': len(top_5_pct_trades)
        }
        
    else:
        print("\n⚠ No trades found in top 5% liquidity")
        top_5_metrics = {}
        top_5_equity_df = pd.DataFrame()
else:
    print("\n⚠ No trades data available")
    top_5_metrics = {}
    top_5_equity_df = pd.DataFrame()

print(f"\n{'='*80}")

In [ ]:
# ============================================================================
# STEP 10: Summary & Key Visualizations
# ============================================================================

print("\n" + "="*80)
print("BACKTEST SUMMARY & KEY INSIGHTS")
print("="*80)

if not trades_df.empty and len(equity_df) > 0:
    print(f"\n📋 STRATEGY CONFIGURATION:")
    print(f"  Market Universe:     Climate markets")
    print(f"  Time Period:         {equity_df['date'].iloc[0]} to {equity_df['date'].iloc[-1]}")
    print(f"  Filters Applied:     Duration >= {MIN_DURATION_HOURS/24:.1f} days, Liquidity >= ${MIN_LIQUIDITY_DOLLARS:,}")
    print(f"  Entry Timing:        12 hours before market close")
    print(f"  Position Sizing:     70% YES on favorite, 30% NO on others")
    print(f"  Daily Budget:        ${DAILY_BUDGET:,}")
    print(f"  Initial Capital:     ${INITIAL_CAPITAL:,}")
    
    print(f"\n🎯 KEY RESULTS:")
    print(f"  Final Equity:        ${equity_df['equity'].iloc[-1]:>12,.2f}")
    print(f"  Total Return:        {performance_metrics.get('total_return_pct', 0):>12.2f}%")
    print(f"  CAGR:                {performance_metrics.get('cagr', 0):>12.2f}%")
    print(f"  Sharpe Ratio:        {performance_metrics.get('sharpe_ratio', 0):>12.2f}")
    print(f"  Max Drawdown:        {performance_metrics.get('max_drawdown', 0):>12.2f}%")
    print(f"  Win Rate:            {performance_metrics.get('win_rate', 0):>12.1f}%")
    print(f"  Total Trades:        {performance_metrics.get('total_trades', 0):>12,}")
    
    print(f"\n📊 MONTHLY BREAKDOWN:")
    # Add month/year to trades
    trades_df['month'] = pd.to_datetime(trades_df['close_date']).dt.to_period('M')
    monthly_pnl = trades_df.groupby('month')['pnl'].sum()
    monthly_trades = trades_df.groupby('month').size()
    
    print(f"  {'Month':<12} {'Trades':>8} {'P&L':>12} {'Cum. Return':>12}")
    print(f"  {'-'*50}")
    
    cumulative_pnl = 0
    for month in monthly_pnl.index:
        cumulative_pnl += monthly_pnl[month]
        cum_return_pct = (cumulative_pnl / INITIAL_CAPITAL) * 100
        print(f"  {str(month):<12} {monthly_trades[month]:>8,} ${monthly_pnl[month]:>11,.2f} {cum_return_pct:>11.2f}%")
    
    print(f"\n📈 BEST & WORST DAYS:")
    best_days = equity_df.nlargest(5, 'daily_pnl')[['date', 'daily_pnl', 'equity']]
    worst_days = equity_df.nsmallest(5, 'daily_pnl')[['date', 'daily_pnl', 'equity']]
    
    print(f"\n  Best 5 Days:")
    print(f"  {'Date':<12} {'Daily P&L':>12} {'Equity':>14}")
    print(f"  {'-'*40}")
    for _, row in best_days.iterrows():
        print(f"  {str(row['date']):<12} ${row['daily_pnl']:>11,.2f} ${row['equity']:>13,.2f}")
    
    print(f"\n  Worst 5 Days:")
    print(f"  {'Date':<12} {'Daily P&L':>12} {'Equity':>14}")
    print(f"  {'-'*40}")
    for _, row in worst_days.iterrows():
        print(f"  {str(row['date']):<12} ${row['daily_pnl']:>11,.2f} ${row['equity']:>13,.2f}")
    
    print(f"\n💡 KEY INSIGHTS:")
    
    # Insight 1: Overall profitability
    if performance_metrics.get('total_return_pct', 0) > 0:
        print(f"  ✓ Strategy is profitable with {performance_metrics.get('total_return_pct', 0):.2f}% total return")
    else:
        print(f"  ✗ Strategy is unprofitable with {performance_metrics.get('total_return_pct', 0):.2f}% total return")
    
    # Insight 2: Risk-adjusted returns
    sharpe = performance_metrics.get('sharpe_ratio', 0)
    if sharpe > 1:
        print(f"  ✓ Excellent risk-adjusted returns (Sharpe = {sharpe:.2f})")
    elif sharpe > 0.5:
        print(f"  ~ Moderate risk-adjusted returns (Sharpe = {sharpe:.2f})")
    else:
        print(f"  ✗ Poor risk-adjusted returns (Sharpe = {sharpe:.2f})")
    
    # Insight 3: Win rate vs expectancy
    win_rate = performance_metrics.get('win_rate', 0)
    if win_rate > 55:
        print(f"  ✓ High win rate ({win_rate:.1f}%) indicates good market selection")
    elif win_rate > 45:
        print(f"  ~ Moderate win rate ({win_rate:.1f}%)")
    else:
        print(f"  ✗ Low win rate ({win_rate:.1f}%) - strategy struggles to predict outcomes")
    
    # Insight 4: Top 5% liquidity comparison
    if top_5_metrics:
        top_5_sharpe = top_5_metrics.get('sharpe_ratio', 0)
        overall_sharpe = performance_metrics.get('sharpe_ratio', 0)
        if top_5_sharpe > overall_sharpe:
            diff = top_5_sharpe - overall_sharpe
            print(f"  ✓ Top 5% liquidity markets outperform (Sharpe +{diff:.2f})")
        else:
            diff = overall_sharpe - top_5_sharpe
            print(f"  ~ Lower liquidity markets perform better (Sharpe +{diff:.2f})")
    
    # Insight 5: Max drawdown analysis
    max_dd = abs(performance_metrics.get('max_drawdown', 0))
    if max_dd < 10:
        print(f"  ✓ Low max drawdown ({max_dd:.1f}%) - capital is well protected")
    elif max_dd < 20:
        print(f"  ~ Moderate max drawdown ({max_dd:.1f}%)")
    else:
        print(f"  ✗ High max drawdown ({max_dd:.1f}%) - significant volatility")
    
    print(f"\n💾 DATA EXPORTS:")
    print(f"  trades_df          - All {len(trades_df)} trades with details")
    print(f"  equity_df          - Daily equity curve")
    print(f"  performance_metrics - Overall strategy metrics")
    print(f"  top_5_metrics      - Top 5% liquidity markets metrics")
    print(f"  top_5_equity_df    - Top 5% equity curve")

else:
    print("\n⚠ WARNING: No backtest data available!")
    print("  This likely means no climate markets met the filtering criteria.")
    print("  Consider:")
    print("    - Lowering liquidity threshold (currently ${MIN_LIQUIDITY_DOLLARS:,})")
    print("    - Shortening duration requirement (currently {MIN_DURATION_HOURS/24:.1f} days)")
    print("    - Broadening market universe beyond climate")

print(f"\n{'='*80}")
print("BACKTEST COMPLETE!")
print("="*80)

# Companies Markets Backtesting Strategy

## Strategy Overview
- **Universe**: All companies-related markets from Kalshi ([Companies Category](https://kalshi.com/category/companies))
- **Constraints**: 
  - Market duration: MIN 1 week (7 days), MAX 1 month (30 days)
  - Liquidity: Adaptive threshold (starting at $100k, lowering if needed)
- **Entry**: 12 hours before market expiration
- **Position Sizing**:
  - Daily budget: $10,000 split across all qualifying events
  - Per event: 70% on YES favorite, 30% split across NO positions
- **Initial Capital**: $100,000
- **Timeframe**: Past year

## Performance Metrics
- Day-by-day equity curve
- Cumulative returns and Sharpe ratio
- Maximum drawdown analysis
- Separate analysis for top 5% liquidity markets

In [35]:
# ============================================================================
# COMPANIES BACKTEST - STEP 1: Pull All Historical Companies Markets
# ============================================================================

import requests
import time
from datetime import datetime, timedelta, timezone
import numpy as np
import pandas as pd
try:
    from zoneinfo import ZoneInfo
    ET_TZ = ZoneInfo('America/New_York')
except ImportError:
    ET_TZ = timezone(timedelta(hours=-5))

print("="*80)
print("FETCHING HISTORICAL COMPANIES MARKETS")
print("="*80)

# Calculate time window (past year)
now_comp = datetime.now(timezone.utc)
one_year_ago_comp = now_comp - timedelta(days=365)
min_close_ts_comp = int(one_year_ago_comp.timestamp())
max_close_ts_comp = int(now_comp.timestamp())

print(f"\nTime window:")
print(f"  From: {one_year_ago_comp.strftime('%Y-%m-%d %H:%M:%S UTC')}")
print(f"  To:   {now_comp.strftime('%Y-%m-%d %H:%M:%S UTC')}")

# Search for Companies category series
print("\n" + "-"*80)
print("Fetching Companies category series...")
print("-"*80)

base_url_comp = "https://api.elections.kalshi.com/trade-api/v2"

# Fetch series by category "Companies"
companies_series = []

try:
    response = requests.get(f"{base_url_comp}/series", params={"category": "Companies", "limit": 200})
    if response.status_code == 200:
        series_data = response.json().get('series', [])
        companies_series.extend(series_data)
        print(f"  ✓ Found {len(series_data)} series in Companies category")
    else:
        print(f"  ✗ Error: Status {response.status_code}")
    time.sleep(0.5)
except Exception as e:
    print(f"  ✗ Error fetching Companies category: {e}")

# If category search fails, try keyword-based search
if len(companies_series) == 0:
    print("\n  Category search returned no results. Trying keyword search...")
    
    # Try different company-related keywords
    search_keywords = ["stock", "company", "earnings", "revenue", "nasdaq", "dow", "sp500"]
    
    for keyword in search_keywords:
        try:
            response = requests.get(f"{base_url_comp}/series", params={"limit": 200})
            if response.status_code == 200:
                all_series = response.json().get('series', [])
                # Filter by keyword
                keyword_series = [
                    s for s in all_series 
                    if keyword.lower() in s.get('title', '').lower() 
                    or keyword.lower() in s.get('category', '').lower()
                    or any(keyword.lower() in tag.lower() for tag in s.get('tags', []))
                ]
                companies_series.extend(keyword_series)
                if keyword_series:
                    print(f"    Found {len(keyword_series)} series matching '{keyword}'")
            time.sleep(0.5)
        except Exception as e:
            print(f"    Error searching for '{keyword}': {e}")

# Deduplicate series
unique_companies_series = {s['ticker']: s for s in companies_series}.values()
print(f"\n✓ Total unique companies series: {len(unique_companies_series)}")

# Display found series
if unique_companies_series:
    print("\nCompanies series found:")
    for s in list(unique_companies_series)[:10]:  # Show first 10
        print(f"  - {s['ticker']}: {s['title']}")
        print(f"    Category: {s.get('category', 'N/A')}")
    if len(unique_companies_series) > 10:
        print(f"  ... and {len(unique_companies_series) - 10} more series")

# Store series tickers
companies_series_tickers = [s['ticker'] for s in unique_companies_series]
companies_series_list = list(unique_companies_series)

print(f"\n✓ Will fetch markets for {len(companies_series_tickers)} companies series")
print(f"{'='*80}")

FETCHING HISTORICAL COMPANIES MARKETS

Time window:
  From: 2025-02-01 08:22:01 UTC
  To:   2026-02-01 08:22:01 UTC

--------------------------------------------------------------------------------
Fetching Companies category series...
--------------------------------------------------------------------------------
  ✓ Found 349 series in Companies category

✓ Total unique companies series: 349

Companies series found:
  - KXLEAVEJANGS: Jan GS
    Category: Companies
  - KXUNSEC: UN seceretary
    Category: Companies
  - KXAPPRANKPAID: Paid app ranking
    Category: Entertainment
  - KXAAPLRING: Apple Ring
    Category: Companies
  - KXIRSHEAD: New Head of IRS
    Category: Companies
  - KXWHOACQTIKTOK: Who will Acquire Tiktok
    Category: Companies
  - KXOPENAICEOCHANGE: OpenAI hires another CEO
    Category: Companies
  - KXMINWAGE: Minimum wage hike
    Category: Politics
  - KXPALANTIR: Palantir
    Category: Companies
  - KXREVIEWRELEASEMKBHDFRIEND: MKBHD Friend
    Category: Com

In [36]:
# ============================================================================
# COMPANIES BACKTEST - STEP 2: Fetch All Markets for Companies Series
# ============================================================================

print("\n" + "="*80)
print("FETCHING MARKETS FOR COMPANIES SERIES")
print("="*80)

# Fetch all markets for companies series
all_companies_markets = []

print(f"\nFetching settled markets for {len(companies_series_tickers)} series...")
print("(This may take a few minutes...)\n")

for i, series_ticker in enumerate(companies_series_tickers, 1):
    cursor = None
    series_markets = []
    
    while True:
        params = {
            "series_ticker": series_ticker,
            "status": "settled",  # Only settled markets for backtesting
            "min_close_ts": min_close_ts_comp,
            "max_close_ts": max_close_ts_comp,
            "limit": 1000
        }
        
        if cursor:
            params["cursor"] = cursor
        
        try:
            response = requests.get(f"{base_url_comp}/markets", params=params)
            
            if response.status_code == 429:
                print(f"  Rate limited, waiting 2 seconds...")
                time.sleep(2)
                continue
            
            if response.status_code != 200:
                print(f"  ✗ Error for {series_ticker}: Status {response.status_code}")
                break
            
            data = response.json()
            markets = data.get('markets', [])
            series_markets.extend(markets)
            
            cursor = data.get('cursor')
            if not cursor:
                break
            
            time.sleep(0.3)  # Rate limiting
            
        except Exception as e:
            print(f"  ✗ Error fetching {series_ticker}: {e}")
            break
    
    if series_markets:
        all_companies_markets.extend(series_markets)
        print(f"  [{i}/{len(companies_series_tickers)}] {series_ticker}: {len(series_markets)} markets")
    
    # Progress update every 5 series
    if i % 5 == 0:
        print(f"  Progress: {i}/{len(companies_series_tickers)} series processed, {len(all_companies_markets)} total markets")

print(f"\n{'='*80}")
print(f"✓ TOTAL MARKETS FETCHED: {len(all_companies_markets)}")
print(f"{'='*80}")

# Store raw markets
companies_markets_raw = all_companies_markets


FETCHING MARKETS FOR COMPANIES SERIES

Fetching settled markets for 349 series...
(This may take a few minutes...)

  [1/349] KXLEAVEJANGS: 1 markets
  [4/349] KXAAPLRING: 1 markets
  Progress: 5/349 series processed, 2 total markets
  Rate limited, waiting 2 seconds...
  [7/349] KXOPENAICEOCHANGE: 1 markets
  [8/349] KXMINWAGE: 1 markets
  [10/349] KXREVIEWRELEASEMKBHDFRIEND: 1 markets
  Progress: 10/349 series processed, 5 total markets
  Rate limited, waiting 2 seconds...
  [12/349] KXINDIAX: 1 markets
  [15/349] KXTESLAYL: 1 markets
  Progress: 15/349 series processed, 7 total markets
  [17/349] KXGMECEOCHANGE: 1 markets
  Rate limited, waiting 2 seconds...
  [19/349] KXCOMPANYACTIONYOUTUBE: 6 markets
  Progress: 20/349 series processed, 14 total markets
  [22/349] KXWIKIMEDIA: 1 markets
  Rate limited, waiting 2 seconds...
  Progress: 25/349 series processed, 15 total markets
  [26/349] KXRELOCATENYCFL: 5 markets
  [27/349] KXALTMANEQUITY: 1 markets
  Rate limited, waiting 2 seco

KeyboardInterrupt: 

In [ ]:
# ============================================================================
# COMPANIES BACKTEST - STEP 3: Filter by Duration (1 Week to 1 Month) and Liquidity
# ============================================================================

print("\n" + "="*80)
print("APPLYING FILTERS: DURATION 1 WEEK - 1 MONTH, LIQUIDITY >= $100,000")
print("="*80)

# Target thresholds
MIN_DURATION_HOURS_COMP = 7 * 24    # 1 week = 168 hours
MAX_DURATION_HOURS_COMP = 30 * 24   # 1 month = 720 hours
MIN_LIQUIDITY_DOLLARS_COMP = 100000  # $100,000

# Apply filters
filtered_companies_markets = []
filter_stats_comp = {
    'total': len(companies_markets_raw),
    'duration_pass': 0,
    'liquidity_pass': 0,
    'both_pass': 0,
    'too_short': 0,
    'too_long': 0,
    'liquidity_fail': 0
}

for market in companies_markets_raw:
    # Parse timestamps
    try:
        open_time = datetime.fromisoformat(market['open_time'].replace('Z', '+00:00'))
        close_time = datetime.fromisoformat(market['close_time'].replace('Z', '+00:00'))
        duration_hours = (close_time - open_time).total_seconds() / 3600
        
        liquidity_dollars = float(market.get('liquidity_dollars', '0'))
        
        # Track filter pass rates
        duration_ok = MIN_DURATION_HOURS_COMP <= duration_hours <= MAX_DURATION_HOURS_COMP
        liquidity_ok = liquidity_dollars >= MIN_LIQUIDITY_DOLLARS_COMP
        
        if duration_ok:
            filter_stats_comp['duration_pass'] += 1
        else:
            if duration_hours < MIN_DURATION_HOURS_COMP:
                filter_stats_comp['too_short'] += 1
            else:
                filter_stats_comp['too_long'] += 1
        
        if liquidity_ok:
            filter_stats_comp['liquidity_pass'] += 1
        else:
            filter_stats_comp['liquidity_fail'] += 1
        
        # Both filters must pass
        if duration_ok and liquidity_ok:
            market['duration_hours'] = duration_hours
            market['liquidity_dollars_float'] = liquidity_dollars
            filtered_companies_markets.append(market)
            filter_stats_comp['both_pass'] += 1
            
    except Exception as e:
        print(f"  Error processing market {market.get('ticker', 'unknown')}: {e}")
        continue

print(f"\nFilter Results:")
print(f"  Total markets:                {filter_stats_comp['total']}")
print(f"  Duration filter (1w - 1m):    {filter_stats_comp['duration_pass']} passed")
print(f"    - Too short (< 1 week):     {filter_stats_comp['too_short']}")
print(f"    - Too long (> 1 month):     {filter_stats_comp['too_long']}")
print(f"  Liquidity filter (>=$100k):   {filter_stats_comp['liquidity_pass']} passed, {filter_stats_comp['liquidity_fail']} failed")
print(f"  ✓ Both filters passed:        {filter_stats_comp['both_pass']} markets")

# If no markets pass strict filters, try lower liquidity thresholds
if filter_stats_comp['both_pass'] == 0:
    print("\n" + "⚠"*40)
    print("⚠ NO MARKETS PASSED STRICT FILTERS")
    print("⚠ Lowering liquidity threshold to find available data...")
    print("⚠"*40)
    
    # Try progressively lower liquidity thresholds (keep duration constant)
    liquidity_thresholds_to_try = [50000, 25000, 10000, 5000, 0]
    
    for min_liq in liquidity_thresholds_to_try:
        filtered_companies_markets = []
        for market in companies_markets_raw:
            try:
                open_time = datetime.fromisoformat(market['open_time'].replace('Z', '+00:00'))
                close_time = datetime.fromisoformat(market['close_time'].replace('Z', '+00:00'))
                duration_hours = (close_time - open_time).total_seconds() / 3600
                liquidity_dollars = float(market.get('liquidity_dollars', '0'))
                
                # Keep duration constraint, lower liquidity
                if MIN_DURATION_HOURS_COMP <= duration_hours <= MAX_DURATION_HOURS_COMP and liquidity_dollars >= min_liq:
                    market['duration_hours'] = duration_hours
                    market['liquidity_dollars_float'] = liquidity_dollars
                    filtered_companies_markets.append(market)
            except:
                continue
        
        if len(filtered_companies_markets) > 0:
            print(f"\n✓ Found {len(filtered_companies_markets)} markets with:")
            print(f"  - Duration: 1 week - 1 month")
            print(f"  - Liquidity >= ${min_liq:,}")
            MIN_LIQUIDITY_DOLLARS_COMP = min_liq
            break
    
    if len(filtered_companies_markets) == 0:
        print("\n⚠ WARNING: No companies markets found even with lowered thresholds!")
        print("Showing sample of available markets:\n")
        for i, m in enumerate(companies_markets_raw[:5], 1):
            try:
                open_time = datetime.fromisoformat(m['open_time'].replace('Z', '+00:00'))
                close_time = datetime.fromisoformat(m['close_time'].replace('Z', '+00:00'))
                duration = (close_time - open_time).total_seconds() / 3600
                liq = float(m.get('liquidity_dollars', '0'))
                print(f"  {i}. {m['ticker']}")
                print(f"     Duration: {duration/24:.2f} days, Liquidity: ${liq:,.2f}")
            except:
                pass

print(f"\n{'='*80}")
print(f"✓ QUALIFIED MARKETS: {len(filtered_companies_markets)}")
print(f"{'='*80}")

# Show sample of qualified markets
if filtered_companies_markets:
    print("\nSample qualified markets (first 5):")
    for i, m in enumerate(filtered_companies_markets[:5], 1):
        print(f"\n  {i}. {m['ticker']}")
        print(f"     Title: {m.get('title', 'N/A')}")
        print(f"     Duration: {m['duration_hours']/24:.2f} days ({m['duration_hours']/24/7:.2f} weeks)")
        print(f"     Liquidity: ${m['liquidity_dollars_float']:,.2f}")
        print(f"     Open: {m['open_time']}")
        print(f"     Close: {m['close_time']}")
        print(f"     Result: {m.get('result', 'N/A')}")

companies_markets_qualified = filtered_companies_markets

In [ ]:
# ============================================================================
# COMPANIES BACKTEST - STEP 4: Group by Event & Calculate Entry Points
# ============================================================================

print("\n" + "="*80)
print("GROUPING MARKETS BY EVENT & CALCULATING ENTRY POINTS")
print("="*80)

# Group markets by event_ticker to identify multi-decision markets
events_dict_comp = {}

for market in companies_markets_qualified:
    event_ticker = market.get('event_ticker', market['ticker'])
    
    if event_ticker not in events_dict_comp:
        events_dict_comp[event_ticker] = []
    
    # Calculate entry point (12 hours before close)
    close_time = datetime.fromisoformat(market['close_time'].replace('Z', '+00:00'))
    entry_time = close_time - timedelta(hours=12)
    
    # Add entry info to market
    market['entry_time'] = entry_time
    market['entry_time_str'] = entry_time.isoformat()
    market['close_time_dt'] = close_time
    
    events_dict_comp[event_ticker].append(market)

print(f"\n✓ Found {len(events_dict_comp)} unique events")

# Classify events
event_classification_comp = {
    'single': [],
    'multi': []
}

for event_ticker, markets in events_dict_comp.items():
    if len(markets) == 1:
        event_classification_comp['single'].append(event_ticker)
    else:
        event_classification_comp['multi'].append(event_ticker)

print(f"\nEvent Classification:")
print(f"  Single-outcome events: {len(event_classification_comp['single'])}")
print(f"  Multi-outcome events:  {len(event_classification_comp['multi'])}")

# Multi-decision events for strategy
multi_decision_events_comp = {k: v for k, v in events_dict_comp.items() if len(v) >= 2}

print(f"\n✓ Multi-decision events for strategy: {len(multi_decision_events_comp)}")

# Show sample multi-decision events
if multi_decision_events_comp:
    print("\nSample multi-decision events (first 3):")
    for i, (event_ticker, markets) in enumerate(list(multi_decision_events_comp.items())[:3], 1):
        print(f"\n  {i}. Event: {event_ticker}")
        print(f"     Number of markets: {len(markets)}")
        print(f"     Entry time: {markets[0]['entry_time'].strftime('%Y-%m-%d %H:%M:%S UTC')}")
        print(f"     Close time: {markets[0]['close_time_dt'].strftime('%Y-%m-%d %H:%M:%S UTC')}")
        print(f"     Markets:")
        for m in markets[:3]:  # Show first 3 markets
            print(f"       - {m['ticker']}: {m.get('title', 'N/A')[:50]}...")
            print(f"         Yes bid/ask: ${m.get('yes_bid', 0)/100:.2f}/${m.get('yes_ask', 0)/100:.2f}")
            print(f"         Result: {m.get('result', 'N/A')}")
        if len(markets) > 3:
            print(f"       ... and {len(markets) - 3} more markets")

# All tradeable events
all_tradeable_events_comp = events_dict_comp.copy()

print(f"\n✓ Total tradeable events (single + multi): {len(all_tradeable_events_comp)}")
print(f"{'='*80}")

In [ ]:
# ============================================================================
# COMPANIES BACKTEST - STEP 5: Organize Events by Entry Date
# ============================================================================

print("\n" + "="*80)
print("ORGANIZING EVENTS BY ENTRY DATE")
print("="*80)

# Organize events by entry date
events_by_entry_date_comp = {}

for event_ticker, markets in all_tradeable_events_comp.items():
    entry_time = markets[0]['entry_time']
    entry_date = entry_time.date()
    
    if entry_date not in events_by_entry_date_comp:
        events_by_entry_date_comp[entry_date] = []
    
    events_by_entry_date_comp[entry_date].append({
        'event_ticker': event_ticker,
        'markets': markets,
        'entry_time': entry_time,
        'num_markets': len(markets)
    })

# Sort dates
sorted_dates_comp = sorted(events_by_entry_date_comp.keys())

print(f"\n✓ Trading spans {len(sorted_dates_comp)} unique dates")
if sorted_dates_comp:
    print(f"  First entry date: {sorted_dates_comp[0]}")
    print(f"  Last entry date:  {sorted_dates_comp[-1]}")

# Events per day statistics
events_per_day_comp = [len(events) for events in events_by_entry_date_comp.values()]
if events_per_day_comp:
    print(f"\nEvents per day statistics:")
    print(f"  Average: {np.mean(events_per_day_comp):.2f}")
    print(f"  Median:  {np.median(events_per_day_comp):.0f}")
    print(f"  Max:     {np.max(events_per_day_comp)}")
    print(f"  Min:     {np.min(events_per_day_comp)}")

# Sample daily schedule
if sorted_dates_comp:
    print(f"\nSample daily schedule (first 5 days):")
    for i, date in enumerate(sorted_dates_comp[:5], 1):
        events = events_by_entry_date_comp[date]
        print(f"\n  Day {i}: {date}")
        print(f"  Events to trade: {len(events)}")
        for event in events[:3]:
            print(f"    - {event['event_ticker']}: {event['num_markets']} markets")
        if len(events) > 3:
            print(f"    ... and {len(events) - 3} more events")

print(f"\n{'='*80}")

In [ ]:
# ============================================================================
# COMPANIES BACKTEST - STEP 6: Simulate Trading Strategy
# ============================================================================

print("\n" + "="*80)
print("SIMULATING TRADING STRATEGY")
print("="*80)

# Strategy parameters (same as climate backtest)
INITIAL_CAPITAL_COMP = 100000
DAILY_BUDGET_COMP = 10000
YES_ALLOCATION_COMP = 0.70
NO_ALLOCATION_COMP = 0.30

all_trades_comp = []
trade_id_comp = 0

print(f"\nStrategy Parameters:")
print(f"  Initial capital:  ${INITIAL_CAPITAL_COMP:,}")
print(f"  Daily budget:     ${DAILY_BUDGET_COMP:,}")
print(f"  YES allocation:   {YES_ALLOCATION_COMP*100:.0f}%")
print(f"  NO allocation:    {NO_ALLOCATION_COMP*100:.0f}% (split across NOs)")

print(f"\n{'-'*80}")
print("Processing trades...")
print("-"*80)

# Process each trading day
for date in sorted_dates_comp:
    events_today = events_by_entry_date_comp[date]
    num_events = len(events_today)
    
    if num_events == 0:
        continue
    
    budget_per_event = DAILY_BUDGET_COMP / num_events
    
    for event in events_today:
        event_ticker = event['event_ticker']
        markets = event['markets']
        num_markets = len(markets)
        
        if num_markets >= 2:
            # Multi-decision: YES on favorite, NO on others
            favorite = max(markets, key=lambda m: m.get('yes_bid', 0))
            others = [m for m in markets if m != favorite]
            
            yes_size_dollars = budget_per_event * YES_ALLOCATION_COMP
            no_size_dollars = budget_per_event * NO_ALLOCATION_COMP
            no_size_per_market = no_size_dollars / len(others) if others else 0
            
            # YES on favorite
            yes_price = favorite.get('yes_ask', favorite.get('yes_bid', 50))
            yes_contracts = (yes_size_dollars * 100) / yes_price if yes_price > 0 else 0
            
            if favorite.get('result') == 'yes':
                yes_pnl = yes_contracts * (100 - yes_price) / 100
            elif favorite.get('result') == 'no':
                yes_pnl = -yes_size_dollars
            elif favorite.get('result') == 'void':
                yes_pnl = 0
            else:
                yes_pnl = 0
            
            trade_id_comp += 1
            all_trades_comp.append({
                'trade_id': trade_id_comp,
                'event_ticker': event_ticker,
                'ticker': favorite['ticker'],
                'entry_date': date,
                'entry_time': event['entry_time'],
                'close_time': favorite['close_time_dt'],
                'side': 'YES',
                'price': yes_price / 100,
                'contracts': yes_contracts,
                'investment': yes_size_dollars,
                'result': favorite.get('result', 'unknown'),
                'pnl': yes_pnl,
                'market_type': 'multi_favorite',
                'liquidity': favorite.get('liquidity_dollars_float', 0),
                'duration_days': favorite.get('duration_hours', 0) / 24
            })
            
            # NO on others
            for other in others:
                no_price = other.get('no_ask', other.get('no_bid', 50))
                no_contracts = (no_size_per_market * 100) / no_price if no_price > 0 else 0
                
                if other.get('result') == 'no':
                    no_pnl = no_contracts * (100 - no_price) / 100
                elif other.get('result') == 'yes':
                    no_pnl = -no_size_per_market
                elif other.get('result') == 'void':
                    no_pnl = 0
                else:
                    no_pnl = 0
                
                trade_id_comp += 1
                all_trades_comp.append({
                    'trade_id': trade_id_comp,
                    'event_ticker': event_ticker,
                    'ticker': other['ticker'],
                    'entry_date': date,
                    'entry_time': event['entry_time'],
                    'close_time': other['close_time_dt'],
                    'side': 'NO',
                    'price': no_price / 100,
                    'contracts': no_contracts,
                    'investment': no_size_per_market,
                    'result': other.get('result', 'unknown'),
                    'pnl': no_pnl,
                    'market_type': 'multi_other',
                    'liquidity': other.get('liquidity_dollars_float', 0),
                    'duration_days': other.get('duration_hours', 0) / 24
                })
        
        else:
            # Single binary market
            market = markets[0]
            yes_bid = market.get('yes_bid', 50)
            
            if yes_bid >= 50:
                yes_price = market.get('yes_ask', yes_bid)
                contracts = (budget_per_event * 100) / yes_price if yes_price > 0 else 0
                
                if market.get('result') == 'yes':
                    pnl = contracts * (100 - yes_price) / 100
                elif market.get('result') == 'no':
                    pnl = -budget_per_event
                elif market.get('result') == 'void':
                    pnl = 0
                else:
                    pnl = 0
                
                side = 'YES'
                price = yes_price
            else:
                no_price = market.get('no_ask', 100 - yes_bid)
                contracts = (budget_per_event * 100) / no_price if no_price > 0 else 0
                
                if market.get('result') == 'no':
                    pnl = contracts * (100 - no_price) / 100
                elif market.get('result') == 'yes':
                    pnl = -budget_per_event
                elif market.get('result') == 'void':
                    pnl = 0
                else:
                    pnl = 0
                
                side = 'NO'
                price = no_price
            
            trade_id_comp += 1
            all_trades_comp.append({
                'trade_id': trade_id_comp,
                'event_ticker': event_ticker,
                'ticker': market['ticker'],
                'entry_date': date,
                'entry_time': event['entry_time'],
                'close_time': market['close_time_dt'],
                'side': side,
                'price': price / 100,
                'contracts': contracts,
                'investment': budget_per_event,
                'result': market.get('result', 'unknown'),
                'pnl': pnl,
                'market_type': 'binary',
                'liquidity': market.get('liquidity_dollars_float', 0),
                'duration_days': market.get('duration_hours', 0) / 24
            })

print(f"\n✓ Simulation complete!")
print(f"  Total trades executed: {len(all_trades_comp)}")
print(f"  Trading days: {len(sorted_dates_comp)}")
print(f"{'='*80}")

trades_df_comp = pd.DataFrame(all_trades_comp)

if not trades_df_comp.empty:
    print(f"\nSample trades (first 10):")
    print(trades_df_comp[['trade_id', 'ticker', 'side', 'price', 'investment', 'result', 'pnl']].head(10).to_string(index=False))